Important steps to do before running the code blocks below!! 🚀🚀
1. Create a data folder and upload your imdb csv data files into it.
2. Create a pieline folder to store metadata from the various components

In [1]:
!mkdir ./data
!mkdir ./pipeline

In [2]:
!pip install tfx

     |████████████████████████████████| 2.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.8 MB 40.2 MB/s 
     |████████████████████████████████| 422 kB 70.8 MB/s 
     |████████████████████████████████| 205 kB 72.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.0 MB/s 
     |████████████████████████████████| 98 kB 8.2 MB/s 
     |████████████████████████████████| 147 kB 63.7 MB/s 
     |████████████████████████████████| 1.7 MB 51.6 MB/s 
     |████████████████████████████████| 1.4 MB 56.0 MB/s 
     |████████████████████████████████| 1.7 MB 40.7 MB/s 
     |████████████████████████████████| 40 kB 5.7 MB/s 
     |████████████████████████████████| 19.1 MB 386 kB/s 
     |████████████████████████████████| 135 kB 63.8 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 47.3 MB/s 
     |████████████████████████████████| 9.9 MB 27.0 MB/s 
     |████████████████████████████████| 2.3 MB 60.4 MB/s 
     |███████████████

In [1]:
!pip install dask[complete]

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/comm

In [43]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tensorflow as tf
import os
import pprint
import dask.dataframe as dd

pp = pprint.PrettyPrinter()


In [44]:
context = InteractiveContext(pipeline_root='pipeline')

# 1. Data Cleaning with Dask 

In [45]:
df1 = dd.read_csv('data/*.csv')
df1.head()

,username,rating,helpful,total,date,title,review
0,bob the moo,Null,109,124,13 October 2002,"Funny, lively and enjoyable despite being a little predictable\n","On his first day at a new school, Cameron falls for Bianca Stratford. However not only is she popular and beautiful but her father has forbidden her to date until her older sister does too. The problem is her sister, Kat, is negative, bad tempered and unfriendly and certainly undateable. Cameron seeks the services of school bad boy Pat who he arranges to be paid to date Kat and thus allow him to go out with Bianca - but things never go smoothly when it comes to love.<br/><br/>I'm not a big fan of teen comedies. They mostly are lame, trading on the same old plots and clichés that have been around since Animal House and Porkies. The recent trend has just seen classical texts adapted giving rise to some poor films like `She's all that'. The plot for this is basically `The Taming of the Shrew' updated - most of it bares little resemblance to the play but that doesn't matter - the core is interesting enough to stand on it's own. It may not surprise you in terms of who gets which girl and it's fairly predictable in a romantic comedy type way but that's not it.<br/><br/>No - the film is funny, lively and pretty enjoyable. The comedy is built around the central plot with lots of nice touches and characters that are genuinely funny, such Mr Morgan and the erotic drama writing headmistress. The jokes happily never stoop to the gross-out type and this helps it feel fresh and lively, rather than just scrapping the bottom of the barrel.<br/><br/>Another strength is that the cast are all very likeable and give good performances all round. Compared to the horror or Prince and Lillard in `She's all that' this was a dream cast. Ledger is a real up and comer and he does well here - he doesn't over-egg his bad boy cake and his fall into love is believable. Stiles is very cool and better than the teen roles she has had of late. Here she rises above the `outsider' cliché given to her and does good work - again being a likeable and believable character.. Gordon-Levitt is OK but quite characterless compared to the main two roles, likewise Oleynik. The bets roles are those on the edges and the three adult support roles really do well and are very funny - Janney, Mitchell and especially Miller on good form. Gabrielle Union is once again given a side role that doesn't play to her strength. I like her and want to see her in bigger roles but here she is a `minority face' in a predominately white film, just as she was in She's All That - maybe I'm being a little paranoid but that's what it feels like.<br/><br/>Overall I was taken by surprise as to how funny and lively this film was. It didn't sink to the gross out comedy of so many teen movies and had characters and actors who brought spark and energy to the screen. It may be predictable but it works and the jokes and side characters just add to the sense of fun. Not brilliant but compared to the mediocre bunch of teen comedies you COULD watch, this is fantastic!"
1,sirokai,10,190,224,27 November 2004,I can't stop watching this movie!\n,"I wanted to see this movie when it first came out but never did. I didn't get a copy until a week ago and finally watched it with my girlfriend. I have to admit, I've seen it five times in the past week. A lot of teen movies have very basic or clichéd plots, but not this one. It's based on ""The Taming of the Shrew"" and it has a lot of interesting twists. The music is awesome, every song actually fits the moment, and the two love stories are actually interesting to watch. Even all the actors did a really great job. The only bad thing is the occasional dirty jokes. If you don't care for teen movies, don't worry, you are just like me, this movie is still worth watching. I'm giving it a 10 to bring those ratings up!"
2,ILoveDMB,9,151,178,5 April 1999,"Sweet *teenage love, lies, & consequences* flick!\n","I have to sa

In [46]:
#dropping unnecessary columns from the dataset
df2 = df1[['review', 'rating']]
df2.head()

,review,rating
0,"On his first day at a new school, Cameron falls for Bianca Stratford. However not only is she popular and beautiful but her father has forbidden her to date until her older sister does too. The problem is her sister, Kat, is negative, bad tempered and unfriendly and certainly undateable. Cameron seeks the services of school bad boy Pat who he arranges to be paid to date Kat and thus allow him to go out with Bianca - but things never go smoothly when it comes to love.<br/><br/>I'm not a big fan of teen comedies. They mostly are lame, trading on the same old plots and clichés that have been around since Animal House and Porkies. The recent trend has just seen classical texts adapted giving rise to some poor films like `She's all that'. The plot for this is basically `The Taming of the Shrew' updated - most of it bares little resemblance to the play but that doesn't matter - the core is interesting enough to stand on it's own. It may not surprise you in terms of who gets which girl and it's fairly predictable in a romantic comedy type way but that's not it.<br/><br/>No - the film is funny, lively and pretty enjoyable. The comedy is built around the central plot with lots of nice touches and characters that are genuinely funny, such Mr Morgan and the erotic drama writing headmistress. The jokes happily never stoop to the gross-out type and this helps it feel fresh and lively, rather than just scrapping the bottom of the barrel.<br/><br/>Another strength is that the cast are all very likeable and give good performances all round. Compared to the horror or Prince and Lillard in `She's all that' this was a dream cast. Ledger is a real up and comer and he does well here - he doesn't over-egg his bad boy cake and his fall into love is believable. Stiles is very cool and better than the teen roles she has had of late. Here she rises above the `outsider' cliché given to her and does good work - again being a likeable and believable character.. Gordon-Levitt is OK but quite characterless compared to the main two roles, likewise Oleynik. The bets roles are those on the edges and the three adult support roles really do well and are very funny - Janney, Mitchell and especially Miller on good form. Gabrielle Union is once again given a side role that doesn't play to her strength. I like her and want to see her in bigger roles but here she is a `minority face' in a predominately white film, just as she was in She's All That - maybe I'm being a little paranoid but that's what it feels like.<br/><br/>Overall I was taken by surprise as to how funny and lively this film was. It didn't sink to the gross out comedy of so many teen movies and had characters and actors who brought spark and energy to the screen. It may be predictable but it works and the jokes and side characters just add to the sense of fun. Not brilliant but compared to the mediocre bunch of teen comedies you COULD watch, this is fantastic!",Null
1,"I wanted to see this movie when it first came out but never did. I didn't get a copy until a week ago and finally watched it with my girlfriend. I have to admit, I've seen it five times in the past week. A lot of teen movies have very basic or clichéd plots, but not this one. It's based on ""The Taming of the Shrew"" and it has a lot of interesting twists. The music is awesome, every song actually fits the moment, and the two love stories are actually interesting to watch. Even all the actors did a really great job. The only bad thing is the occasional dirty jokes. If you don't care for teen movies, don't worry, you are just like me, this movie is still worth watching. I'm giving it a 10 to bring those ratings up!",10
2,"I have to say, I thoroughly enjoyed this movie. Granted I'm just 15 years old, and also a huge fan of Shakespeare, but it was really a great film. I would have to say it's definitely more directed towards the teenage/GenX generations, but it was awesome. The tongue-and-cheek Shakespeare references were 

In [47]:
#drop all rows with the null rating
df3 = df2[df2.rating != 'Null']
df3.head()

,review,rating
1,"I wanted to see this movie when it first came out but never did. I didn't get a copy until a week ago and finally watched it with my girlfriend. I have to admit, I've seen it five times in the past week. A lot of teen movies have very basic or clichéd plots, but not this one. It's based on ""The Taming of the Shrew"" and it has a lot of interesting twists. The music is awesome, every song actually fits the moment, and the two love stories are actually interesting to watch. Even all the actors did a really great job. The only bad thing is the occasional dirty jokes. If you don't care for teen movies, don't worry, you are just like me, this movie is still worth watching. I'm giving it a 10 to bring those ratings up!",10
2,"I have to say, I thoroughly enjoyed this movie. Granted I'm just 15 years old, and also a huge fan of Shakespeare, but it was really a great film. I would have to say it's definitely more directed towards the teenage/GenX generations, but it was awesome. The tongue-and-cheek Shakespeare references were great (the ""Stratford"" girls, etc.) Julia Stiles & Larisa Oleynik were convincing in their roles, and not to be shallow but the guys of the movie just all looked really good. =) I recommend this to most teenage girls, although it's not totally a ""chick flick"" because my brother enjoyed it too. In any case, the scene with the ""Can't Take My Eyes Off You"" ballad at the soccer practice is a can't-miss for all you love saps. =) I will tell you right now, the ending IS predictable, but not hackneyed. It leaves you with a happy bubbly feeling about teenage love. Excellent movie, that is. (P.S. When watching this in the theaters-- DON'T leave as soon as the credits roll. There are some hilarious out-takes afterwards.)",9
3,"A movie like this doesn't exactly hit you over the head with its originality, and can only be as good as the actors. They must convince you that you've never seen anything like this before. In this case every single member of the cast is so good that they sold me on the movie from minute one. Heath Ledger & Julia Stiles make a great couple and his rendition of ""Can't Take My Eyes Off of You"" was one of the highlights. When Julia listed the ""10 Things"" she was crying and I was, too - extremely unusual for me in a movie like this. Everyone else was just as good and this movie is a keeper.",9
4,"Love love LOVE this movie. It is a classic, story is great, the actors are amazing, and the comedy is funny! There is something to laugh at every minute, and all the elements marry together perfectly to make a feel good movie, with depth and humour that isn't seen very often these days.<br/><br/>The cast are brilliant, Heath Ledger is awesome and brings an honesty to his character, Julia stiles is authentic and someone every girl can identify with. Has a lot of actors who made it big after the film - great cast.<br/><br/>An old school movie, they don't make them like this anymore!",10
6,"All the praise in the world goes to every individual involved in the production of 10 Things I Hate About You,a film that rarely leaves my DVD player.I'm not a fan of Shakespeare's traditional plays,but the modern day screen adaptions are brilliant,and 10 Things I Hate About You is no exception.The highschool stereotypes of the cool kids nobody messes with,the wannabe's that try too hard to be something they're not,the geeks everyone picks on and the rebels everyone is afraid of,mean there's a character in this film everyone can relate to.The stand out performances are Heath Ledger as Patrick,the rebellious,feared and elusive bad guy who wins the heart of Kat Stratford played to perfection by fine actress Julia Stiles who has no interest in the opposite sex and is perceived to come from the planet 'Loser' while her more sociable,boy crazy sister Bianca (Larissa Oleynik)hails from planet 'Look at Me,Look at Me'.Her good looks catch the eye of the smooth,suave and egotistical Joey Donner (Andrew Keegan

In [48]:
#Change all ratings to sentiment types
#ratings are initially strings so we need to change them to integers

df3.rating = df3.rating.astype('int')
df3.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 2 entries, review to rating
dtypes: object(1), int64(1)

In [49]:
df3['sentiment'] = (df3.rating > 5).astype('int')
df3.head()

,review,rating,sentiment
1,"I wanted to see this movie when it first came out but never did. I didn't get a copy until a week ago and finally watched it with my girlfriend. I have to admit, I've seen it five times in the past week. A lot of teen movies have very basic or clichéd plots, but not this one. It's based on ""The Taming of the Shrew"" and it has a lot of interesting twists. The music is awesome, every song actually fits the moment, and the two love stories are actually interesting to watch. Even all the actors did a really great job. The only bad thing is the occasional dirty jokes. If you don't care for teen movies, don't worry, you are just like me, this movie is still worth watching. I'm giving it a 10 to bring those ratings up!",10,1
2,"I have to say, I thoroughly enjoyed this movie. Granted I'm just 15 years old, and also a huge fan of Shakespeare, but it was really a great film. I would have to say it's definitely more directed towards the teenage/GenX generations, but it was awesome. The tongue-and-cheek Shakespeare references were great (the ""Stratford"" girls, etc.) Julia Stiles & Larisa Oleynik were convincing in their roles, and not to be shallow but the guys of the movie just all looked really good. =) I recommend this to most teenage girls, although it's not totally a ""chick flick"" because my brother enjoyed it too. In any case, the scene with the ""Can't Take My Eyes Off You"" ballad at the soccer practice is a can't-miss for all you love saps. =) I will tell you right now, the ending IS predictable, but not hackneyed. It leaves you with a happy bubbly feeling about teenage love. Excellent movie, that is. (P.S. When watching this in the theaters-- DON'T leave as soon as the credits roll. There are some hilarious out-takes afterwards.)",9,1
3,"A movie like this doesn't exactly hit you over the head with its originality, and can only be as good as the actors. They must convince you that you've never seen anything like this before. In this case every single member of the cast is so good that they sold me on the movie from minute one. Heath Ledger & Julia Stiles make a great couple and his rendition of ""Can't Take My Eyes Off of You"" was one of the highlights. When Julia listed the ""10 Things"" she was crying and I was, too - extremely unusual for me in a movie like this. Everyone else was just as good and this movie is a keeper.",9,1
4,"Love love LOVE this movie. It is a classic, story is great, the actors are amazing, and the comedy is funny! There is something to laugh at every minute, and all the elements marry together perfectly to make a feel good movie, with depth and humour that isn't seen very often these days.<br/><br/>The cast are brilliant, Heath Ledger is awesome and brings an honesty to his character, Julia stiles is authentic and someone every girl can identify with. Has a lot of actors who made it big after the film - great cast.<br/><br/>An old school movie, they don't make them like this anymore!",10,1
6,"All the praise in the world goes to every individual involved in the production of 10 Things I Hate About You,a film that rarely leaves my DVD player.I'm not a fan of Shakespeare's traditional plays,but the modern day screen adaptions are brilliant,and 10 Things I Hate About You is no exception.The highschool stereotypes of the cool kids nobody messes with,the wannabe's that try too hard to be something they're not,the geeks everyone picks on and the rebels everyone is afraid of,mean there's a character in this film everyone can relate to.The stand out performances are Heath Ledger as Patrick,the rebellious,feared and elusive bad guy who wins the heart of Kat Stratford played to perfection by fine actress Julia Stiles who has no interest in the opposite sex and is perceived to come from the planet 'Loser' while her more sociable,boy crazy sister Bianca (Larissa Oleynik)hails from planet 'Look at Me,Look at Me'.Her good looks catch the eye of the smooth,suave and egotistical Joey Don

In [50]:
#convert dataframe to csv
df4 = df3[['review', 'sentiment']]
df4.to_csv('./data_in.csv', index=False)

['/content/./data_in.csv/00.part',
 '/content/./data_in.csv/01.part',
 '/content/./data_in.csv/02.part',
 '/content/./data_in.csv/03.part',
 '/content/./data_in.csv/04.part',
 '/content/./data_in.csv/05.part',
 '/content/./data_in.csv/06.part',
 '/content/./data_in.csv/07.part',
 '/content/./data_in.csv/08.part',
 '/content/./data_in.csv/09.part',
 '/content/./data_in.csv/10.part',
 '/content/./data_in.csv/11.part',
 '/content/./data_in.csv/12.part',
 '/content/./data_in.csv/13.part',
 '/content/./data_in.csv/14.part',
 '/content/./data_in.csv/15.part',
 '/content/./data_in.csv/16.part',
 '/content/./data_in.csv/17.part',
 '/content/./data_in.csv/18.part',
 '/content/./data_in.csv/19.part',
 '/content/./data_in.csv/20.part',
 '/content/./data_in.csv/21.part',
 '/content/./data_in.csv/22.part',
 '/content/./data_in.csv/23.part',
 '/content/./data_in.csv/24.part',
 '/content/./data_in.csv/25.part',
 '/content/./data_in.csv/26.part',
 '/content/./data_in.csv/27.part',
 '/content/./data_in

In [51]:
df4.head()

,review,sentiment
1,"I wanted to see this movie when it first came out but never did. I didn't get a copy until a week ago and finally watched it with my girlfriend. I have to admit, I've seen it five times in the past week. A lot of teen movies have very basic or clichéd plots, but not this one. It's based on ""The Taming of the Shrew"" and it has a lot of interesting twists. The music is awesome, every song actually fits the moment, and the two love stories are actually interesting to watch. Even all the actors did a really great job. The only bad thing is the occasional dirty jokes. If you don't care for teen movies, don't worry, you are just like me, this movie is still worth watching. I'm giving it a 10 to bring those ratings up!",1
2,"I have to say, I thoroughly enjoyed this movie. Granted I'm just 15 years old, and also a huge fan of Shakespeare, but it was really a great film. I would have to say it's definitely more directed towards the teenage/GenX generations, but it was awesome. The tongue-and-cheek Shakespeare references were great (the ""Stratford"" girls, etc.) Julia Stiles & Larisa Oleynik were convincing in their roles, and not to be shallow but the guys of the movie just all looked really good. =) I recommend this to most teenage girls, although it's not totally a ""chick flick"" because my brother enjoyed it too. In any case, the scene with the ""Can't Take My Eyes Off You"" ballad at the soccer practice is a can't-miss for all you love saps. =) I will tell you right now, the ending IS predictable, but not hackneyed. It leaves you with a happy bubbly feeling about teenage love. Excellent movie, that is. (P.S. When watching this in the theaters-- DON'T leave as soon as the credits roll. There are some hilarious out-takes afterwards.)",1
3,"A movie like this doesn't exactly hit you over the head with its originality, and can only be as good as the actors. They must convince you that you've never seen anything like this before. In this case every single member of the cast is so good that they sold me on the movie from minute one. Heath Ledger & Julia Stiles make a great couple and his rendition of ""Can't Take My Eyes Off of You"" was one of the highlights. When Julia listed the ""10 Things"" she was crying and I was, too - extremely unusual for me in a movie like this. Everyone else was just as good and this movie is a keeper.",1
4,"Love love LOVE this movie. It is a classic, story is great, the actors are amazing, and the comedy is funny! There is something to laugh at every minute, and all the elements marry together perfectly to make a feel good movie, with depth and humour that isn't seen very often these days.<br/><br/>The cast are brilliant, Heath Ledger is awesome and brings an honesty to his character, Julia stiles is authentic and someone every girl can identify with. Has a lot of actors who made it big after the film - great cast.<br/><br/>An old school movie, they don't make them like this anymore!",1
6,"All the praise in the world goes to every individual involved in the production of 10 Things I Hate About You,a film that rarely leaves my DVD player.I'm not a fan of Shakespeare's traditional plays,but the modern day screen adaptions are brilliant,and 10 Things I Hate About You is no exception.The highschool stereotypes of the cool kids nobody messes with,the wannabe's that try too hard to be something they're not,the geeks everyone picks on and the rebels everyone is afraid of,mean there's a character in this film everyone can relate to.The stand out performances are Heath Ledger as Patrick,the rebellious,feared and elusive bad guy who wins the heart of Kat Stratford played to perfection by fine actress Julia Stiles who has no interest in the opposite sex and is perceived to come from the planet 'Loser' while her more sociable,boy crazy sister Bianca (Larissa Oleynik)hails from planet 'Look at Me,Look at Me'.Her good looks catch the eye of the smooth,suave and egotistical Joey Donner (Andrew Keega

# 2. Pipeline

## 2.1 Data Ingestion

## 2.1.1 ExampleGen

In [52]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
        ])
)

In [53]:
#!mv data/sample_data/ ./

In [54]:
example_gen = CsvExampleGen(input_base='data_in.csv', output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 6
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 13
        type_id: 14
        uri: "pipeline/CsvExampleGen/examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:73,total_bytes:33830003,xor_checksum:1641801406,sum_checksum:119851502616"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [55]:
train_url = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')
#eval_url = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-eval')

In [56]:
tfrecord_filenames = [os.path.join(train_url, name) for name in os.listdir(train_url)]
#ev_tfrecord_filenames = [os.path.join(eval_url, name) for name in os.listdir(eval_url)]

In [57]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')
#ds_eval = tf.data.TFRecordDataset(ev_tfrecord_filenames, compression_type='GZIP')

In [58]:
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)


features {
  feature {
    key: "review"
    value {
      bytes_list {
        value: "After writing a phd dissertation and spending months doing research on the Alamo at The DRT library and across Texas, I became convinced that I might not live long enough to see a theatrical release that would finally do the history justice. The IMAX Alamo film is very good along historical lines, but due to budget limitations not to mention physiological IMAX constraints, it did not capture the scope and depth of the event. But make no mistake about it - this Alamo film does both. The Alamo\'s major participants are three dimensional flesh and blood mirror images of those one will find in their diaries, letters, books and first hand accounts of those who knew them. Even the Mexican dictator, Antonio Lopez Miguel De Santa Anna, is no longer a cardboard demon - he anticipates what Mexico will become without the stern hand that must come down to crush \"the American pirates.\" Also, for the first time

In [59]:
#for tfrecord in ds_eval.take(2):
#  ev_serialized_example = tfrecord.numpy()
#  ev_example = tf.train.Example()
#  ev_example.ParseFromString(ev_serialized_example)
#  pp.pprint(ev_example)

# 2.2 Data Validation

## 2.2.1 StatisticsGen

In [60]:
from tfx.components import StatisticsGen


In [61]:
statistics_gen = StatisticsGen(
    examples= example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 7
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 14
        type_id: 16
        uri: "pipeline/StatisticsGen/statistics/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [62]:
context.show(statistics_gen.outputs['statistics'])

## 2.2.2 SchemaGen

In [63]:
from tfx.components import SchemaGen

In [64]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

In [65]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 8
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 15
        type_id: 18
        uri: "pipeline/SchemaGen/schema/8"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [66]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'review',BYTES,required,,-
'sentiment',INT,required,,-


## 2.2.3 ExampleValidator

In [67]:
from tfx.components import ExampleValidator

In [68]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = schema_gen.outputs['schema']
)

In [69]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 9
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 16
        type_id: 20
        uri: "pipeline/ExampleValidator/anomalies/9"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [70]:
context.show(example_validator.outputs['anomalies'])

# 2.3 Data Preprocessing

## 2.3.1 Transform

NLP tasks need the following transforms
- change all the text to lowercase
- remove html tags
- stop words
- remove punctuations

In [71]:
_transform_module_file = '_transform.py'

In [81]:
%%writefile {_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft


stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", 
             "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", 
             "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", 
             "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", 
             "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", 
             "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", 
             "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", 
             "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", 
             "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", 
             "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", 
             "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

_LABEL_KEY = 'sentiment'

#Renaming all transform features
def _transformed_name(key):
  return key + '_xf'

#Define the transformations. The name is fixed since that is what is expected
def preprocessing_fn(inputs):
  outputs = {}
  #Change all the texts to lowercase
  outputs[_transformed_name('review')] = tf.strings.lower(inputs['review'])
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], r'(?:<br/>)', ' ')
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], 'n\'t', ' not ')
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], r'(?:\'ll |\'re |\'d |\'ve|\'s)', ' ')
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], r'\W+', ' ')
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], r'\d+', ' ')
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], r'(\b[a-zA-Z]\b)', ' ')
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], r'\b(' + r'|'.join(stopwords) + r')\b\s*', ' ')
  outputs[_transformed_name('review')] = tf.strings.regex_replace(outputs[_transformed_name('review')], r'\s\s+', ' ')
  outputs[_transformed_name(_LABEL_KEY)] = tf.cast(inputs[_LABEL_KEY], tf.int64)


  return outputs


Overwriting _transform.py


In [73]:
#run the transform component
from tfx.components import Transform

transform = Transform(
    examples= example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=_transform_module_file
)


In [74]:
context.run(transform)

INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/10/.temp_path/tftransform_tmp/66f2bc4ccc984792ba26952102418ef1/assets


INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/10/.temp_path/tftransform_tmp/66f2bc4ccc984792ba26952102418ef1/assets


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 17
        type_id: 22
        uri: "pipeline/Transform/transform_graph/10"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 18
        type_id: 14
        uri: "pipeline/Transform/transformed_examples/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"eval\", \"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 19
        type_id: 23
        uri: "pipeline/Transform/updated_analyzer_cache/10"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 20
        type_id: 18
        uri: "pipeline/Transform/pre_transform_schema/10"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
   

Showing the transformed outputs

In [75]:
train_url = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
tfrecord_filenames = [os.path.join(train_url, name) for name in os.listdir(train_url)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "review_xf"
    value {
      bytes_list {
        value: " writing phd dissertation spending months research alamo drt library across texas became convinced might live long enough see theatrical release would finally history justice imax alamo film good along historical lines due budget limitations mention physiological imax constraints capture scope depth event make mistake alamo film alamo major participants three dimensional flesh blood mirror images one find diaries letters books first hand accounts knew even mexican dictator antonio lopez miguel de santa anna longer cardboard demon anticipates mexico become without stern hand must come crush american pirates also first time tejanos fought brothers sisters texas cause well represented battle sequences culled santa anna battle plans accounts carried survived leave nuance imagination vividly demonstrate even chaotic retreat turn unmanageable enemy force overwhelming west north walls alamo bloodbath fu

# 2.4 Model Training

## 2.4.1 Tuner
We would implement this portion on cloud platforms not on local servers.

## 2.4.2 Trainer

In [76]:
#Declare a trainer_module_file
_trainer_module_file = '_trainer.py'

In [88]:
%%writefile {_trainer_module_file}

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

_LABEL_KEY = 'sentiment'
_FEATURE = 'review'

def _transformed_name(key):
  return key + '_xf'

def _gzip_reader_fn(filenames):
  ''' Loads compressed data '''
  return tf.data.TFRecordDataset(filenames=filenames, compression_type='GZIP')

def _input_fn(file_pattern, 
              tf_transform_output, 
              num_epochs, 
              batch_size=64)-> tf.data.Dataset:
  # Get our post-transform feature specification
  transform_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy()
  )

  #Create batches of the data
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transform_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=_transformed_name(_LABEL_KEY)
  )
  return dataset
  
def model_builder():
  embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4") #universal sentence encoder for embedding

  inputs = tf.keras.Input(shape=(1,), name=_transformed_name('review'), dtype=tf.string)
  reshaped_narrative = tf.reshape(inputs, [-1])
  x = embed(reshaped_narrative)
  x= tf.keras.layers.Reshape((1,512), input_shape=(1,512))(x)
  x = layers.Dense(64, activation='elu', kernel_initializer='glorot_uniform')(x)

  attn_output = layers.MultiHeadAttention(num_heads=2, key_dim=64)(x, x, x)
  attn_output = layers.Dropout(rate=0.2)(attn_output)

  out1 = layers.LayerNormalization(epsilon=1e-7)(x + attn_output)
  ffn_output = layers.Dense(64, activation='elu', kernel_initializer='glorot_uniform')(out1)
  ffn_output = layers.Dropout(rate=0.2)(ffn_output)

  x= layers.LayerNormalization(epsilon=1e-7)(out1 + ffn_output)
  x= layers.GlobalAveragePooling1D()(x)
  x = layers.Dropout(rate=0.2)(x)
  x = layers.Dense(32, activation='elu', kernel_initializer='glorot_uniform')(x)
  x = layers.Dropout(rate=0.2)(x)

  outputs = layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.Adam(0.01),
      metrics=[tf.keras.metrics.BinaryAccuracy()]
  )

  #print model
  model.summary()
  
  return model

def run_fn(fn_args:FnArgs) -> None:
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir = fn_args.model_run_dir, update_freq='batch'
  )
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
  model_checkpoint = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)

  #Load the transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  #create batches of data
  train_set = _input_fn(fn_args.train_files, tf_transform_output, 10)
  val_set = _input_fn(fn_args.eval_files, tf_transform_output, 10)

  #Build the model
  model = model_builder()

  #Train the model
  model.fit(x=train_set,
            validation_data=val_set,
            callbacks = [tensorboard_callback, early_stopping, model_checkpoint],
            epochs=100)






Overwriting _trainer.py


In [ ]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])

)

context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 review_xf (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 tf.reshape_1 (TFOpLambda)      (None,)              0           ['review_xf[0][0]']              
                                                                                                  
 keras_layer_1 (KerasLayer)     (None, 512)          256797824   ['tf.reshape_1[0][0]']           
                                                                                                  
 reshape_1 (Reshape)            (None, 1, 512)       0           ['keras_layer_1[0][0]']          
                                                                                            

INFO:tensorflow:Assets written to: pipeline/Trainer/model/14/Format-Serving/assets


INFO:tensorflow:Assets written to: pipeline/Trainer/model/14/Format-Serving/assets


3300/3300 [==============================] - 302s 90ms/step - loss: 0.3530 - binary_accuracy: 0.8492 - val_loss: 0.3489 - val_binary_accuracy: 0.8655
Epoch 2/100
1983/3300 [=================>............] - ETA: 1:30 - loss: 0.3279 - binary_accuracy: 0.8588